# Character-Level LSTM in PyTorch

In this notebook, represent LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, It will be trained on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn).

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# 1. int2char, maps integers to characters
# 2. char2int, maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
print(int2char.items())

# encode the text
encoded = np.array([char2int[ch] for ch in text])

dict_items([(0, 'C'), (1, '4'), (2, 'Q'), (3, 't'), (4, '2'), (5, 'X'), (6, 'R'), (7, 'l'), (8, '*'), (9, '0'), (10, 'x'), (11, '('), (12, 'v'), (13, 'h'), (14, '-'), (15, 'I'), (16, 'U'), (17, '_'), (18, 'J'), (19, '@'), (20, '$'), (21, '\n'), (22, '5'), (23, 'g'), (24, ')'), (25, 'c'), (26, 'f'), (27, 'a'), (28, 's'), (29, '.'), (30, 'Y'), (31, '?'), (32, '3'), (33, '/'), (34, 'w'), (35, 'A'), (36, 'D'), (37, 'H'), (38, 'F'), (39, 'o'), (40, '8'), (41, '7'), (42, '`'), (43, 'G'), (44, '!'), (45, '1'), (46, ','), (47, 'W'), (48, 'n'), (49, 'E'), (50, '&'), (51, "'"), (52, 'Z'), (53, ';'), (54, 'k'), (55, 'q'), (56, 'u'), (57, '"'), (58, 'L'), (59, '9'), (60, 'd'), (61, ' '), (62, '6'), (63, 'i'), (64, 'y'), (65, 'j'), (66, ':'), (67, 'r'), (68, 'B'), (69, 'S'), (70, 'N'), (71, 'p'), (72, 'e'), (73, 'z'), (74, 'V'), (75, 'P'), (76, 'M'), (77, 'T'), (78, 'K'), (79, 'b'), (80, '%'), (81, 'O'), (82, 'm')])


## Encoded text

In [5]:
encoded[:100]

array([ 0, 13, 27, 71,  3, 72, 67, 61, 45, 21, 21, 21, 37, 27, 71, 71, 64,
       61, 26, 27, 82, 63,  7, 63, 72, 28, 61, 27, 67, 72, 61, 27,  7,  7,
       61, 27,  7, 63, 54, 72, 53, 61, 72, 12, 72, 67, 64, 61, 56, 48, 13,
       27, 71, 71, 64, 61, 26, 27, 82, 63,  7, 64, 61, 63, 28, 61, 56, 48,
       13, 27, 71, 71, 64, 61, 63, 48, 61, 63,  3, 28, 61, 39, 34, 48, 21,
       34, 27, 64, 29, 21, 21, 49, 12, 72, 67, 64,  3, 13, 63, 48])

## Pre-processing the data
Network exepts **one-hot encoded characters**

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches

Before training data will be divided into mini-batches.
<br>

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches that can be made
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Implementation

Now I'll make some data sets to check out what's going on with batch data. Here, as an example, with batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])

# data should be shifted over one step for y
print('\ny\n', y[:10, :10])

x
 [[ 0 13 27 71  3 72 67 61 45 21]
 [28 39 48 61  3 13 27  3 61 27]
 [72 48 60 61 39 67 61 27 61 26]
 [28 61  3 13 72 61 25 13 63 72]
 [61 28 27 34 61 13 72 67 61  3]
 [25 56 28 28 63 39 48 61 27 48]
 [61 35 48 48 27 61 13 27 60 61]
 [81 79  7 39 48 28 54 64 29 61]]

y
 [[13 27 71  3 72 67 61 45 21 21]
 [39 48 61  3 13 27  3 61 27  3]
 [48 60 61 39 67 61 27 61 26 39]
 [61  3 13 72 61 25 13 63 72 26]
 [28 27 34 61 13 72 67 61  3 72]
 [56 28 28 63 39 48 61 27 48 60]
 [35 48 48 27 61 13 27 60 61 28]
 [79  7 39 48 28 54 64 29 61 57]]


---
## Defining the network with PyTorch

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```                           
`input_size` is the number of characters this cell expects to see as sequential input
`n_hidden` is the number of units in the hidden layers in the cell    
`n_layers` is the number of layers    
`dropout_prob` is the dropout probability

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ##  Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ##  pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Training

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

In [14]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [15]:
batch_size = 128
seq_length = 10
n_epochs = 27 # start smaller just to testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/27... Step: 10... Loss: 3.2473... Val Loss: 3.2347
Epoch: 1/27... Step: 20... Loss: 3.1577... Val Loss: 3.1467
Epoch: 1/27... Step: 30... Loss: 3.1138... Val Loss: 3.1346
Epoch: 1/27... Step: 40... Loss: 3.1067... Val Loss: 3.1283
Epoch: 1/27... Step: 50... Loss: 3.1263... Val Loss: 3.1276
Epoch: 1/27... Step: 60... Loss: 3.1081... Val Loss: 3.1238
Epoch: 1/27... Step: 70... Loss: 3.0536... Val Loss: 3.1232
Epoch: 1/27... Step: 80... Loss: 3.1011... Val Loss: 3.1213
Epoch: 1/27... Step: 90... Loss: 3.0660... Val Loss: 3.1112
Epoch: 1/27... Step: 100... Loss: 3.0696... Val Loss: 3.0904
Epoch: 1/27... Step: 110... Loss: 2.9959... Val Loss: 3.0407
Epoch: 1/27... Step: 120... Loss: 2.9881... Val Loss: 2.9539
Epoch: 1/27... Step: 130... Loss: 2.9240... Val Loss: 2.9243
Epoch: 1/27... Step: 140... Loss: 2.7555... Val Loss: 2.8232
Epoch: 1/27... Step: 150... Loss: 2.6804... Val Loss: 2.7172
Epoch: 1/27... Step: 160... Loss: 2.6114... Val Loss: 2.6272
Epoch: 1/27... Step: 170... Loss:

Epoch: 1/27... Step: 1350... Loss: 1.6281... Val Loss: 1.6388
Epoch: 1/27... Step: 1360... Loss: 1.5561... Val Loss: 1.6339
Epoch: 1/27... Step: 1370... Loss: 1.5816... Val Loss: 1.6347
Epoch: 1/27... Step: 1380... Loss: 1.7323... Val Loss: 1.6306
Epoch: 1/27... Step: 1390... Loss: 1.6063... Val Loss: 1.6256
Epoch: 2/27... Step: 1400... Loss: 1.6051... Val Loss: 1.6303
Epoch: 2/27... Step: 1410... Loss: 1.5341... Val Loss: 1.6229
Epoch: 2/27... Step: 1420... Loss: 1.6312... Val Loss: 1.6279
Epoch: 2/27... Step: 1430... Loss: 1.5922... Val Loss: 1.6187
Epoch: 2/27... Step: 1440... Loss: 1.5881... Val Loss: 1.6225
Epoch: 2/27... Step: 1450... Loss: 1.6041... Val Loss: 1.6116
Epoch: 2/27... Step: 1460... Loss: 1.5796... Val Loss: 1.6134
Epoch: 2/27... Step: 1470... Loss: 1.5394... Val Loss: 1.6151
Epoch: 2/27... Step: 1480... Loss: 1.6056... Val Loss: 1.6099
Epoch: 2/27... Step: 1490... Loss: 1.6614... Val Loss: 1.6075
Epoch: 2/27... Step: 1500... Loss: 1.5308... Val Loss: 1.6076
Epoch: 2

Epoch: 2/27... Step: 2680... Loss: 1.3975... Val Loss: 1.4717
Epoch: 2/27... Step: 2690... Loss: 1.3562... Val Loss: 1.4736
Epoch: 2/27... Step: 2700... Loss: 1.4403... Val Loss: 1.4745
Epoch: 2/27... Step: 2710... Loss: 1.4769... Val Loss: 1.4704
Epoch: 2/27... Step: 2720... Loss: 1.2886... Val Loss: 1.4718
Epoch: 2/27... Step: 2730... Loss: 1.5179... Val Loss: 1.4657
Epoch: 2/27... Step: 2740... Loss: 1.4060... Val Loss: 1.4675
Epoch: 2/27... Step: 2750... Loss: 1.3578... Val Loss: 1.4811
Epoch: 2/27... Step: 2760... Loss: 1.3188... Val Loss: 1.4699
Epoch: 2/27... Step: 2770... Loss: 1.3725... Val Loss: 1.4720
Epoch: 2/27... Step: 2780... Loss: 1.3863... Val Loss: 1.4708
Epoch: 2/27... Step: 2790... Loss: 1.9398... Val Loss: 1.4647
Epoch: 3/27... Step: 2800... Loss: 1.4492... Val Loss: 1.4854
Epoch: 3/27... Step: 2810... Loss: 1.4151... Val Loss: 1.4635
Epoch: 3/27... Step: 2820... Loss: 1.4288... Val Loss: 1.4685
Epoch: 3/27... Step: 2830... Loss: 1.3938... Val Loss: 1.4616
Epoch: 3

Epoch: 3/27... Step: 4010... Loss: 1.3342... Val Loss: 1.4110
Epoch: 3/27... Step: 4020... Loss: 1.3569... Val Loss: 1.4151
Epoch: 3/27... Step: 4030... Loss: 1.3089... Val Loss: 1.4111
Epoch: 3/27... Step: 4040... Loss: 1.2851... Val Loss: 1.4156
Epoch: 3/27... Step: 4050... Loss: 1.3324... Val Loss: 1.4093
Epoch: 3/27... Step: 4060... Loss: 1.3426... Val Loss: 1.4071
Epoch: 3/27... Step: 4070... Loss: 1.3001... Val Loss: 1.4112
Epoch: 3/27... Step: 4080... Loss: 1.3848... Val Loss: 1.4072
Epoch: 3/27... Step: 4090... Loss: 1.3807... Val Loss: 1.4072
Epoch: 3/27... Step: 4100... Loss: 1.4162... Val Loss: 1.4146
Epoch: 3/27... Step: 4110... Loss: 1.3378... Val Loss: 1.4039
Epoch: 3/27... Step: 4120... Loss: 1.3104... Val Loss: 1.4179
Epoch: 3/27... Step: 4130... Loss: 1.3966... Val Loss: 1.4212
Epoch: 3/27... Step: 4140... Loss: 1.3502... Val Loss: 1.4040
Epoch: 3/27... Step: 4150... Loss: 1.2844... Val Loss: 1.4090
Epoch: 3/27... Step: 4160... Loss: 1.3388... Val Loss: 1.4236
Epoch: 3

Epoch: 4/27... Step: 5340... Loss: 1.2837... Val Loss: 1.3774
Epoch: 4/27... Step: 5350... Loss: 1.2760... Val Loss: 1.3812
Epoch: 4/27... Step: 5360... Loss: 1.2719... Val Loss: 1.3790
Epoch: 4/27... Step: 5370... Loss: 1.2939... Val Loss: 1.3851
Epoch: 4/27... Step: 5380... Loss: 1.2993... Val Loss: 1.3768
Epoch: 4/27... Step: 5390... Loss: 1.2159... Val Loss: 1.3775
Epoch: 4/27... Step: 5400... Loss: 1.2190... Val Loss: 1.3813
Epoch: 4/27... Step: 5410... Loss: 1.3668... Val Loss: 1.3774
Epoch: 4/27... Step: 5420... Loss: 1.3092... Val Loss: 1.3792
Epoch: 4/27... Step: 5430... Loss: 1.3086... Val Loss: 1.3792
Epoch: 4/27... Step: 5440... Loss: 1.3275... Val Loss: 1.3774
Epoch: 4/27... Step: 5450... Loss: 1.2832... Val Loss: 1.3731
Epoch: 4/27... Step: 5460... Loss: 1.3485... Val Loss: 1.3823
Epoch: 4/27... Step: 5470... Loss: 1.2736... Val Loss: 1.3728
Epoch: 4/27... Step: 5480... Loss: 1.2391... Val Loss: 1.3720
Epoch: 4/27... Step: 5490... Loss: 1.2979... Val Loss: 1.3784
Epoch: 4

...

Epoch: 23/27... Step: 31310... Loss: 1.1460... Val Loss: 1.3114
Epoch: 23/27... Step: 31320... Loss: 1.1697... Val Loss: 1.3109
Epoch: 23/27... Step: 31330... Loss: 1.1110... Val Loss: 1.3139
Epoch: 23/27... Step: 31340... Loss: 1.0831... Val Loss: 1.3195
Epoch: 23/27... Step: 31350... Loss: 1.1128... Val Loss: 1.3145
Epoch: 23/27... Step: 31360... Loss: 1.0904... Val Loss: 1.3081
Epoch: 23/27... Step: 31370... Loss: 1.1516... Val Loss: 1.3077
Epoch: 23/27... Step: 31380... Loss: 1.1465... Val Loss: 1.3142
Epoch: 23/27... Step: 31390... Loss: 1.1674... Val Loss: 1.3142
Epoch: 23/27... Step: 31400... Loss: 1.0891... Val Loss: 1.3106
Epoch: 23/27... Step: 31410... Loss: 1.1201... Val Loss: 1.3098
Epoch: 23/27... Step: 31420... Loss: 1.0522... Val Loss: 1.3097
Epoch: 23/27... Step: 31430... Loss: 1.0803... Val Loss: 1.3130
Epoch: 23/27... Step: 31440... Loss: 1.1343... Val Loss: 1.3134
Epoch: 23/27... Step: 31450... Loss: 1.1058... Val Loss: 1.3107
Epoch: 23/27... Step: 31460... Loss: 1.0

Epoch: 24/27... Step: 32600... Loss: 1.0990... Val Loss: 1.3043
Epoch: 24/27... Step: 32610... Loss: 1.0996... Val Loss: 1.3036
Epoch: 24/27... Step: 32620... Loss: 1.0646... Val Loss: 1.3052
Epoch: 24/27... Step: 32630... Loss: 1.0499... Val Loss: 1.3120
Epoch: 24/27... Step: 32640... Loss: 1.0414... Val Loss: 1.3161
Epoch: 24/27... Step: 32650... Loss: 1.0384... Val Loss: 1.3127
Epoch: 24/27... Step: 32660... Loss: 1.0393... Val Loss: 1.3085
Epoch: 24/27... Step: 32670... Loss: 1.0911... Val Loss: 1.3095
Epoch: 24/27... Step: 32680... Loss: 1.0532... Val Loss: 1.3143
Epoch: 24/27... Step: 32690... Loss: 1.1075... Val Loss: 1.3140
Epoch: 24/27... Step: 32700... Loss: 1.0686... Val Loss: 1.3108
Epoch: 24/27... Step: 32710... Loss: 1.1164... Val Loss: 1.3104
Epoch: 24/27... Step: 32720... Loss: 1.0599... Val Loss: 1.3102
Epoch: 24/27... Step: 32730... Loss: 1.1664... Val Loss: 1.3126
Epoch: 24/27... Step: 32740... Loss: 1.1175... Val Loss: 1.3177
Epoch: 24/27... Step: 32750... Loss: 1.1

Epoch: 25/27... Step: 33890... Loss: 1.1154... Val Loss: 1.3116
Epoch: 25/27... Step: 33900... Loss: 1.1135... Val Loss: 1.3085
Epoch: 25/27... Step: 33910... Loss: 1.1174... Val Loss: 1.3077
Epoch: 25/27... Step: 33920... Loss: 1.1591... Val Loss: 1.3095
Epoch: 25/27... Step: 33930... Loss: 1.1332... Val Loss: 1.3058
Epoch: 25/27... Step: 33940... Loss: 1.1494... Val Loss: 1.3039
Epoch: 25/27... Step: 33950... Loss: 1.1757... Val Loss: 1.3042
Epoch: 25/27... Step: 33960... Loss: 1.0713... Val Loss: 1.3087
Epoch: 25/27... Step: 33970... Loss: 1.0347... Val Loss: 1.3108
Epoch: 25/27... Step: 33980... Loss: 1.1171... Val Loss: 1.3106
Epoch: 25/27... Step: 33990... Loss: 1.0919... Val Loss: 1.3057
Epoch: 25/27... Step: 34000... Loss: 1.1430... Val Loss: 1.3065
Epoch: 25/27... Step: 34010... Loss: 1.0496... Val Loss: 1.3061
Epoch: 25/27... Step: 34020... Loss: 1.0558... Val Loss: 1.3101
Epoch: 25/27... Step: 34030... Loss: 1.1164... Val Loss: 1.3150
Epoch: 25/27... Step: 34040... Loss: 1.0

Epoch: 26/27... Step: 35180... Loss: 1.0655... Val Loss: 1.3120
Epoch: 26/27... Step: 35190... Loss: 1.0927... Val Loss: 1.3155
Epoch: 26/27... Step: 35200... Loss: 1.0587... Val Loss: 1.3149
Epoch: 26/27... Step: 35210... Loss: 1.0707... Val Loss: 1.3097
Epoch: 26/27... Step: 35220... Loss: 1.0836... Val Loss: 1.3070
Epoch: 26/27... Step: 35230... Loss: 1.0734... Val Loss: 1.3053
Epoch: 26/27... Step: 35240... Loss: 1.1127... Val Loss: 1.3049
Epoch: 26/27... Step: 35250... Loss: 1.1390... Val Loss: 1.3053
Epoch: 26/27... Step: 35260... Loss: 1.0695... Val Loss: 1.3062
Epoch: 26/27... Step: 35270... Loss: 1.0887... Val Loss: 1.3124
Epoch: 26/27... Step: 35280... Loss: 1.1316... Val Loss: 1.3145
Epoch: 26/27... Step: 35290... Loss: 1.0941... Val Loss: 1.3116
Epoch: 26/27... Step: 35300... Loss: 1.0382... Val Loss: 1.3060
Epoch: 26/27... Step: 35310... Loss: 1.1510... Val Loss: 1.3059
Epoch: 26/27... Step: 35320... Loss: 1.0717... Val Loss: 1.3089
Epoch: 26/27... Step: 35330... Loss: 1.0

Epoch: 27/27... Step: 36470... Loss: 1.0573... Val Loss: 1.3056
Epoch: 27/27... Step: 36480... Loss: 1.0441... Val Loss: 1.3064
Epoch: 27/27... Step: 36490... Loss: 1.0905... Val Loss: 1.3103
Epoch: 27/27... Step: 36500... Loss: 1.0850... Val Loss: 1.3074
Epoch: 27/27... Step: 36510... Loss: 1.1027... Val Loss: 1.3064
Epoch: 27/27... Step: 36520... Loss: 1.0297... Val Loss: 1.3045
Epoch: 27/27... Step: 36530... Loss: 1.0164... Val Loss: 1.3061
Epoch: 27/27... Step: 36540... Loss: 0.9933... Val Loss: 1.3077
Epoch: 27/27... Step: 36550... Loss: 1.1102... Val Loss: 1.3088
Epoch: 27/27... Step: 36560... Loss: 1.0813... Val Loss: 1.3079
Epoch: 27/27... Step: 36570... Loss: 1.1390... Val Loss: 1.3091
Epoch: 27/27... Step: 36580... Loss: 1.0921... Val Loss: 1.3146
Epoch: 27/27... Step: 36590... Loss: 1.0571... Val Loss: 1.3164
Epoch: 27/27... Step: 36600... Loss: 1.0983... Val Loss: 1.3113
Epoch: 27/27... Step: 36610... Loss: 1.1031... Val Loss: 1.3076
Epoch: 27/27... Step: 36620... Loss: 1.1

## Checkpoint

Saving model

In [16]:
# change the name, for saving multiple files
model_name = 'rnn.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [23]:
print(sample(net, 1000, prime='train', top_k=5))

train they can study in the world was
the first time had before a look with the child, so that those thought to be a crop when
she was asleep
he had not answered what he was seeing him to anyone that
he was ashamed, the footman
and
this party had time to see him that she was
so satisfied, were in a solemal and the feeling of carriage and the compenit entering a sting, who had still
more, and the sick man had. That would have spent the music and drawing her all these weight, and he did
not see what to do all, his wife had been
carrying at the same time, and he had seen them. He did not say that he diversed the portfolio, and had thruwted her he was not to speak of all that he had arrived, though their
family
she was seriously that he had seen his study, and serious and a merchant's secret clack of his strange thing absorbed him any one of the count, who had
answered shrills on the pitiful position, and he
had been a sense of tone, the future still had so the point of clever her husband 

## Loading a checkpoint

In [20]:
# Here we have loaded in a model
with open('rnn.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [21]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said, and the carriage he had an understand himself, as he went off to the little conception.

"Oh, yes, this woman is so a langer he doesn't say something," he said, sitting down by
her.

"Yes, that's nothing?" she asked herself.
"I was not so left alone, but I'm such a little, and though I can did not think of you."

"Oh, yes, that you
more about it. And what,
if I do not advise you to do, I'm going to
the memaronity and thoughts of it....
I'd better be the second and telling you to
them to blame, and so introduces itself. It's not this, if I would be in subjects of
my life, but when she came to tell her anything
without the steps to her again, and he had
to be dull for the same position; but and he was stupid
indeed and that there was not telling the stream of any opportonity of the same
past of the strange few
masters. He felt that he had
talked
without stayed on at the sick man, he had better never accessing him.

"I'm not given in that woman. What's true to that, but yo